In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
cd drive/My\ Drive/NLP/cnn-text-classification-pytorch/

/content/drive/My Drive/NLP/cnn-text-classification-pytorch


In [6]:
ls

LICENSE   mydatasets.py  rt-polaritydata/                snapshot/
main.py   __pycache__/   rt-polaritydata.README.1.0.txt  train.py
model.py  README.md      rt-polaritydata.tar             try1/


In [0]:
import h5py

In [0]:
path_to_h5py = "../sent-conv-torch/custom.hdf5"
hf = h5py.File(path_to_h5py,'r')

In [0]:
keys = list(hf.keys())

In [0]:
w2v = hf['w2v']
data = hf['train']
label = hf['train_label']

In [0]:
w2v = hf.get('w2v').value

In [0]:

data = hf.get('train').value

In [0]:
label = hf.get('train_label').value

In [0]:
max_len = 0
this = 0
for x in data:
  c = 0
  l = len(x) - 1
  while(x[l]==1 and l>=0):
    c+=1
    l-=1
  c = len(x) - c
  if c > max_len:
    max_len = c
    this = x

In [0]:
max_len

551

In [0]:
for i in range(data.shape[0]):
  for j in range(len(data[i])):
    data[i][j] = data[i][j] - 1

In [0]:
import numpy as np

In [0]:
import torch.nn as nn
import torch

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
ones = []
zeros = []
for i in range(len(label)):
  if label[i]==1:
    ones.append(i)
  else:
    zeros.append(i)

In [0]:
ones = np.array(ones)
zeros = np.array(zeros)

In [0]:
np.random.seed(0)
np.random.shuffle(ones)
np.random.shuffle(zeros)

In [0]:
np.random.seed(0)

split = .8
train_indices = []
val_indices = []
count = 0
for i in range(len(ones)):
  if count<= split*len(ones)+1:
    train_indices.append(ones[i])
    count+=1
  else:
    val_indices.append(ones[i])

count = 0
for i in range(len(zeros)):
  if count<= split*len(zeros) +1:
    train_indices.append(zeros[i])
    count+=1
  else:
    val_indices.append(zeros[i])  

val_indices = np.array(val_indices)
train_indices = np.array(train_indices)
np.random.shuffle(val_indices)
np.random.shuffle(train_indices)
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [27]:
len(train_indices)

10403

In [28]:
len(val_indices)

2597

In [0]:
class text_dataset(torch.utils.data.Dataset):
    
    def __init__(self, data, label):
        # Begin
        # self.train = train  # boolean to check if train or test
        # if train:
        #     self.root_dir = os.path.join(root_dir , 'trainval')
        #     labels_path = os.path.join(self.root_dir, 'labels2.txt')
        #     self.names = [(name.split(" ")[0].strip() , name.split(" ")[1].strip()) for name in open(labels_path)]
        #     self.names = sorted(self.names , key=lambda x: x[1])
        # else:
        #     self.root_dir = os.path.join(root_dir , 'test/VOCdevkit/VOC2007/')
        #     test_path = os.path.join(self.root_dir, 'list.txt')
        #     self.names = [name.strip() for name in open(test_path)]
        #     self.names = sorted(self.names , key=lambda x: x[1])
                    
        
        # self.transform = transform            # transforms to be done 

        self.data = data
        self.labels = label

        
    def __len__(self):
        # Begin
        return len(self.labels)
        
    def __getitem__(self, idx):

        l = torch.tensor(self.labels[idx]).long()
        sentence = torch.tensor(self.data[idx]).long()

        return sentence, l

        # Begin
        # if self.train:
        #     name = self.names[idx][0]
        #     label = classes.index(self.names[idx][1])
        #     image_path = os.path.join(self.root_dir , 'croped2/' + name)  # load image
        #     img = image_loader(image_path)
            
        #     if self.transform is not None:
        #         img = self.transform(img)  # apply transforms
            
        #     return img, label   # return image's tensor and label

        
    # def test_iter(self, idx):    # only for Testing.
    #     '''This function loads test image with index = idx, and return's  it's path and the list of bounding boxes:
    #     [class, x1,y1,x2,y2]'''
        
    #     image_name = self.names[idx]
    #     annot_path = os.path.join(self.root_dir , 'Annotations')
    #     image_path = os.path.join(self.root_dir , 'Images/' + image_name)
    #     img = image_path

    #     tree = ET.parse(os.path.join(annot_path , image_name.split('.')[0] + '.xml'))
    #     root = tree.getroot()
    #     id =0
    #     bounding_boxes=[]
    #     for child in root.iter('object'):
    #         label = []
    #         name = child.find('name').text
    #         if name != 'aeroplane' and name != 'chair' and name != 'bottle':
    #             continue
    #         label.append(classes.index(name))
    #         bndbox = child.find('bndbox')   # load bounding box
    #         label.append(int(bndbox.find('xmin').text))
    #         label.append(int(bndbox.find('ymin').text))
    #         label.append(int(bndbox.find('xmax').text))
    #         label.append(int(bndbox.find('ymax').text))
    #         bounding_boxes.append(np.array(label))

    #     return img, np.array(bounding_boxes)



In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class CNN_Text(nn.Module):        

    def __init__(self, args, weight_matrix):
        super(CNN_Text, self).__init__()
        self.args = args
        self.embed, num_embeddings, embedding_dim = self.create_emb_layer(weight_matrix, args.non_static)

        V = num_embeddings
        D = embedding_dim
        C = args.class_num
        Ci = 1
        Co = args.kernel_num
        Ks = args.kernel_sizes

        # self.convs1 = [nn.Conv2d(Ci, Co, (K, D)) for K in Ks]
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])
        '''
        self.conv13 = nn.Conv2d(Ci, Co, (3, D))
        self.conv14 = nn.Conv2d(Ci, Co, (4, D))
        self.conv15 = nn.Conv2d(Ci, Co, (5, D))
        '''
        self.dropout = nn.Dropout(args.dropout)
        self.fc1 = nn.Linear(len(Ks)*Co, C)
    
    def create_emb_layer(self, weight_matrix, train_weights=False):
        num_embeddings, embedding_dim = weight_matrix.size()
        emb_layer = nn.Embedding(num_embeddings, embedding_dim)
        emb_layer.load_state_dict({'weight': weight_matrix})
        if train_weights:
            emb_layer.weight.requires_grad = True
        else:
            emb_layer.weight.requires_grad = False

        return emb_layer, num_embeddings, embedding_dim

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3)  # (N, Co, W)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        # print(x.size())
        
        if self.args.non_static == False:
            x = Variable(x)

        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        '''
        x1 = self.conv_and_pool(x,self.conv13) #(N,Co)
        x2 = self.conv_and_pool(x,self.conv14) #(N,Co)
        x3 = self.conv_and_pool(x,self.conv15) #(N,Co)
        x = torch.cat((x1, x2, x3), 1) # (N,len(Ks)*Co)
        '''
        x = self.dropout(x)  # (N, len(Ks)*Co)
        logit = self.fc1(x)  # (N, C)
        return logit


In [0]:
import os
import sys
import torch
import torch.autograd as autograd
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


def train(train_iter, dev_iter, model, args):
    if args.cuda:
        model.cuda()
        print("On Cuda")

    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    weights = [1,10]
    weights = torch.FloatTensor(weights).cuda()
    criterion = nn.CrossEntropyLoss(weight=weights)

    steps = 0
    best_acc = 0
    last_step = 0
    to_return = {}
    max_so_far = 0.0
    for epoch in range(1, args.epochs+1):
        model.train()
        train_preds = None
        train_labels = None
        running_loss = 0.0
        total_loss = 0.0
        mini_batch = 0
        print("epoch: ",epoch)
        scheduler.step()

        for feature, labels in train_iter:
            mini_batch+=1
            
            if args.cuda:
                feature, labels = feature.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = model(feature)

            # print('outputs size', outputs.size())
            # print('target vector', target.size())
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            total_loss += loss.item()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            steps += 1
            # if steps % args.log_interval == 0:
                # corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
                # accuracy = 100.0 * corrects/args.batch_size
                # sys.stdout.write(
                #     '\rBatch[{}] - loss: {:.6f}  acc: {:.4f}%({}/{})'.format(steps, 
                #                                                              loss.data, 
                #                                                              accuracy,
                #                                                              corrects,
                #                                                              args.batch_size))
            if train_preds is None or train_labels is None:
              train_preds = np.argmax(outputs.detach().cpu().numpy(), axis=1).flatten()
              train_labels = labels.cpu().numpy().flatten()
            else:
              train_preds = np.concatenate((train_preds, np.argmax(outputs.detach().cpu().numpy(), axis=1).flatten()))
              train_labels = np.concatenate((train_labels, labels.cpu().numpy().flatten()))


            loss.backward()
            optimizer.step()
            # scheduler.step()

            if mini_batch % 100 == 0:    # print every 100 mini-batches
              print('[%d, %5d] loss: %.3f' % (epoch , mini_batch , running_loss / 100))
              running_loss = 0.0
            
            # if steps % args.test_interval == 0:
            #     dev_acc = eval(dev_iter, model, args)
            #     if dev_acc > best_acc:
            #         best_acc = dev_acc
            #         last_step = steps
            #         if args.save_best:
            #             save(model, args.save_dir, 'best', steps)
            #     else:
            #         if steps - last_step >= args.early_stop:
            #             print('early stop by {} steps.'.format(args.early_stop))
            # elif steps % args.save_interval == 0:
            #     save(model, args.save_dir, 'snapshot', steps)
        
        print("Training loss in epoch %d is %.3f" % (epoch , total_loss / len(train_loader)))
        print("Training accuracy in epoch %d is %.3f" % (epoch, accuracy_score(train_labels, train_preds) * 100))
        print("Training precision in epoch %d is %.3f" % (epoch , precision_score(train_labels, train_preds) * 100))
        print("Training recall in epoch %d is %.3f" % (epoch , recall_score(train_labels, train_preds) * 100))
        print("Training F1-score in epoch %d is %.3f" % (epoch, f1_score(train_labels, train_preds) * 100))
        
        eval(dev_iter, model, criterion, epoch, args, to_return, max_so_far)
        print()
        print("-"*90)
        print()
        
    return to_return


def eval(data_iter, model, criterion, epoch, args, to_return, max_so_far):
    
    model.eval()
    test_loss = 0.0
    test_preds = None
    test_labels = None
    test_inputs = None
    # corrects, avg_loss = 0, 0
    with torch.no_grad():

        for data in data_iter:
            inputs, labels = data
            if args.cuda:
                inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
          
            test_loss += loss.item()
            if test_preds is None or test_labels is None:
                test_preds = np.argmax(outputs.detach().cpu().numpy(), axis=1).flatten()
                test_labels = labels.cpu().numpy().flatten()
                test_inputs = inputs.cpu().numpy()
            else:
                test_preds = np.concatenate((test_preds, np.argmax(outputs.detach().cpu().numpy(), axis=1).flatten()))
                test_labels = np.concatenate((test_labels, labels.cpu().numpy().flatten()))
                test_inputs = np.concatenate((test_inputs, inputs.cpu().numpy()), axis = 0)

            # test_accuracy += flat_accuracy(outputs[0], labels)
    
    print("Test loss in epoch %d is %.3f" % (epoch, test_loss / len(val_loader)))
    print("Test accuracy in epoch %d is %.3f" % (epoch, accuracy_score(test_labels, test_preds) * 100))
    print("Test precision in epoch %d is %.3f" % (epoch, precision_score(test_labels, test_preds) * 100))
    print("Test recall in epoch %d is %.3f" % (epoch, recall_score(test_labels, test_preds) * 100))
    print("Test F1-score in epoch %d is %.3f" % (epoch, f1_score(test_labels, test_preds) * 100))
    if f1_score(test_labels, test_preds)*100 > max_so_far:
      max_so_far = f1_score(test_labels, test_preds)*100
      to_return['pred'] = test_preds
      to_return['label'] = test_labels
      to_return['inputs'] = test_inputs

    # for feature, target in data_iter:
    #     # feature, target = batch.text, batch.label
    #     # feature.data.t_(), target.data.sub_(1)  # batch first, index align
    #     if args.cuda:
    #         feature, target = feature.cuda(), target.cuda()

    #     logit = model(feature)
    #     loss = criterion(logit, target)

    #     avg_loss += loss.data
    #     corrects += (torch.max(logit, 1)
    #                  [1].view(target.size()).data == target.data).sum()

    # size = len(data_iter.dataset)*0.2
    # avg_loss /= size
    # accuracy = 100.0 * corrects/size
    # print('\nEvaluation - loss: {:.6f}  acc: {:.4f}%({}/{}) \n'.format(avg_loss, 
    #                                                                    accuracy, 
    #                                                                    corrects, 
    #                                                                    size))


    # return accuracy

    # print("Val loss in epoch %d is %.3f" % (epoch + 1, test_loss / len(test_loader)))
    # print("Val accuracy in epoch %d is %.3f" % (epoch + 1, accuracy_score(test_labels, test_preds) * 100))
    # print("Val precision in epoch %d is %.3f" % (epoch + 1, precision_score(test_labels, test_preds) * 100))
    # print("Val recall in epoch %d is %.3f" % (epoch + 1, recall_score(test_labels, test_preds) * 100))
    # print("Val   F1-score in epoch %d is %.3f" % (epoch + 1, f1_score(test_labels, test_preds) * 100))


def predict(text, model, text_field, label_feild, cuda_flag):
    assert isinstance(text, str)
    model.eval()
    # text = text_field.tokenize(text)
    text = text_field.preprocess(text)
    text = [[text_field.vocab.stoi[x] for x in text]]
    x = torch.tensor(text)
    x = autograd.Variable(x)
    if cuda_flag:
        x = x.cuda()
    print(x)
    output = model(x)
    _, predicted = torch.max(output, 1)
    #return label_feild.vocab.itos[predicted.data[0][0]+1]
    return label_feild.vocab.itos[predicted.data[0]+1]


def save(model, save_dir, save_prefix, steps):
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    save_prefix = os.path.join(save_dir, save_prefix)
    save_path = '{}_steps_{}.pt'.format(save_prefix, steps)
    torch.save(model.state_dict(), save_path)


In [73]:
import os
import argparse
import datetime
import torch
# import torchtext.data as data
# import torchtext.datasets as datasets
# import model
# import train
# import mydatasets


parser = argparse.ArgumentParser(description='CNN text classificer')
# learning
parser.add_argument('-lr', type=float, default=0.001, help='initial learning rate [default: 0.001]')
parser.add_argument('-epochs', type=int, default=15, help='number of epochs for train [default: 256]')
parser.add_argument('-batch-size', type=int, default=32, help='batch size for training [default: 64]')
parser.add_argument('-log-interval',  type=int, default=1,   help='how many steps to wait before logging training status [default: 1]')
parser.add_argument('-test-interval', type=int, default=100, help='how many steps to wait before testing [default: 100]')
parser.add_argument('-save-interval', type=int, default=500, help='how many steps to wait before saving [default:500]')
parser.add_argument('-save-dir', type=str, default='try1', help='where to save the snapshot')
parser.add_argument('-early-stop', type=int, default=1000, help='iteration numbers to stop without performance increasing')
parser.add_argument('-save-best', type=bool, default=True, help='whether to save when get best performance')
# data 
parser.add_argument('-shuffle', action='store_true', default=True, help='shuffle the data every epoch')
# model
parser.add_argument('-dropout', type=float, default=0.5, help='the probability for dropout [default: 0.5]')
parser.add_argument('-max-norm', type=float, default=3.0, help='l2 constraint of parameters [default: 3.0]')
parser.add_argument('-embed-dim''''  ''', type=int, default=300, help='number of embedding dimension [default: 128]')
parser.add_argument('-kernel-num', type=int, default=100, help='number of each kind of kernel')
parser.add_argument('-kernel-sizes', type=str, default='3,4,5', help='comma-separated kernel size to use for convolution')
parser.add_argument('-non_static', action='store_true', default=False, help='fix the embedding')
# device
parser.add_argument('-device', type=int, default=-1, help='device to use for iterate data, -1 mean cpu [default: -1]')
parser.add_argument('-no-cuda', action='store_true', default=False, help='disable the gpu')
# option
parser.add_argument('-snapshot', type=str, default=None, help='filename of model snapshot [default: None]')
parser.add_argument('-predict', type=str, default=None, help='predict the sentence given')
parser.add_argument('-test', action='store_true', default=False, help='train or test')
args = parser.parse_args(args=[])


# load data 
print("\nLoading data...")
# text_field = data.Field(lower=True)
# label_field = data.Field(sequential=False)
# train_iter, dev_iter = mr(text_field, label_field, device=-1, repeat=False)
dataset = text_dataset(data, label)
train_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=args.batch_size,
                                           sampler = train_sampler)
val_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=args.batch_size,
                                           sampler = valid_sampler)

# train_iter, dev_iter, test_iter = sst(text_field, label_field, device=-1, repeat=False)


# update args and print
args.embed_num = weight_matrix.size()[1]
args.class_num = 2 #binary
args.cuda = (not args.no_cuda) and torch.cuda.is_available(); del args.no_cuda
args.kernel_sizes = [int(k) for k in args.kernel_sizes.split(',')]
args.save_dir = os.path.join(args.save_dir, datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

print("\nParameters:")
for attr, value in sorted(args.__dict__.items()):
    print("\t{}={}".format(attr.upper(), value))
print("Training sentences: ", int(len(dataset)*split))
print("Validation sentences: ", int(len(dataset)*(1-split)))



Loading data...

Parameters:
	BATCH_SIZE=32
	CLASS_NUM=2
	CUDA=True
	DEVICE=-1
	DROPOUT=0.5
	EARLY_STOP=1000
	EMBED_DIM  =300
	EMBED_NUM=300
	EPOCHS=15
	KERNEL_NUM=100
	KERNEL_SIZES=[3, 4, 5]
	LOG_INTERVAL=1
	LR=0.001
	MAX_NORM=3.0
	NON_STATIC=False
	PREDICT=None
	SAVE_BEST=True
	SAVE_DIR=try1/2020-02-01_16-12-10
	SAVE_INTERVAL=500
	SHUFFLE=True
	SNAPSHOT=None
	TEST=False
	TEST_INTERVAL=100
Training sentences:  10400
Validation sentences:  2599


In [0]:
# model 
cnn = CNN_Text(args, weight_matrix)
if args.snapshot is not None:
    print('\nLoading model from {}...'.format(args.snapshot))
    cnn.load_state_dict(torch.load(args.snapshot))

if args.cuda:
    torch.cuda.set_device(args.device)
    cnn = cnn.cuda()
        



In [86]:
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
# train or predict
# static
if args.predict is not None:
    label = predict(args.predict, cnn, text_field, label_field, args.cuda)
    print('\n[Text]  {}\n[Label] {}\n'.format(args.predict, label))
elif args.test:
    try:
        eval(test_iter, cnn, args) 
    except Exception as e:
        print("\nSorry. The test dataset doesn't  exist.\n")
else:
    print()
    try:
        val = train(train_loader, val_loader, cnn, args)
    except KeyboardInterrupt:
        print('\n' + '-' * 89)
        print('Exiting from training early')


On Cuda
epoch:  1


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,   100] loss: 0.606
[1,   200] loss: 0.458
[1,   300] loss: 0.471
Training loss in epoch 1 is 0.503
Training accuracy in epoch 1 is 73.267
Training precision in epoch 1 is 26.151
Training recall in epoch 1 is 76.052
Training F1-score in epoch 1 is 38.919
Test loss in epoch 1 is 0.418
Test accuracy in epoch 1 is 86.292
Test precision in epoch 1 is 43.366
Test recall in epoch 1 is 75.779
Test F1-score in epoch 1 is 55.164

------------------------------------------------------------------------------------------

epoch:  2
[2,   100] loss: 0.380
[2,   200] loss: 0.357
[2,   300] loss: 0.337
Training loss in epoch 2 is 0.354
Training accuracy in epoch 2 is 84.774
Training precision in epoch 2 is 41.389
Training recall in epoch 2 is 86.438
Training F1-score in epoch 2 is 55.976
Test loss in epoch 2 is 0.359
Test accuracy in epoch 2 is 84.521
Test precision in epoch 2 is 40.722
Test recall in epoch 2 is 85.813
Test F1-score in epoch 2 is 55.234

------------------------------------------

In [78]:
val['inputs'].shape

(2597, 555)

In [0]:
import pickle

In [0]:
with open('val_set.pickle', 'wb') as f:
  pickle.dump(val, f )

In [81]:
ls

LICENSE        README.md                       train.py
main.py        rt-polaritydata/                try1/
model.py       rt-polaritydata.README.1.0.txt  val_set.pickle
mydatasets.py  rt-polaritydata.tar
__pycache__/   snapshot/


In [0]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

In [0]:
input

tensor([[-1.0909e+00, -4.3674e-01, -1.5669e+00, -1.3722e+00,  2.7839e-01],
        [-3.6084e-01, -1.5469e-01, -3.0640e-04,  4.9972e-01,  1.5021e+00],
        [-8.6501e-01, -9.9743e-01, -8.3489e-01,  9.9229e-01,  6.1073e-01]],
       requires_grad=True)

In [0]:
target

tensor([0, 4, 1])